In [ ]:
#Load libs

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from tysserand import tysserand as ty
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 
from PIL import Image, ImageOps
import fcsparser

#set up working dir

directory = str("/mnt/SERVER-CRCT-STORAGE/CRCT_Imagin/CORDELIER Pierre/HaloData/21-003.IMMCORE.C1v1/Halo archive 2021-12-07 10-09 - v3.3.2541/ObjectData")
outpath = '/home/mouneem/tysserand/Imagin_LungPredict/nets/'

In [ ]:
# FCS TO CSV
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    print(filename)
    if os.path.isfile(f) and 'fcs' in filename:

        meta, data = fcsparser.parse(f, reformat_meta=True)
        type(data)
        #df1 = df[['a', 'b']] // df1 = df.iloc[:, 0:2] 

        data["x"] = (data["XMax"] + data["XMin"] ) / 2
        data["y"] = (data["YMax"] + data["YMin"] ) / 2

        col_is_marker = []
        for col in list(data.columns):
            if col.count('+') + col.count('-') > 1:
                col_is_marker.append(True)
            else:
                col_is_marker.append(False)

        print(len(col_is_marker))

        marker = data.loc[:, col_is_marker]

        classes = marker.apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
        data["class"] = pd.factorize(classes)[0] 

        nodes = data[['x','y',"class"]]
        nodes.to_csv(outpath + "../csv/"+filename )

        ratio = 50
        nodes = nodes.sample(n = round(len(nodes) / ratio) )
        coords = nodes.loc[:,['x','y']].values

        plt.scatter(coords[:,0] , coords[:,1] ,   zorder=10, s = 1)
    
        plt.savefig(outpath+filename+'.png')
        
        plt.cla()